In [13]:
from importlib import reload
import numpy as np
from matplotlib import pyplot as plt
import Two_qubits_design_PP_qubits as tqd
import QCreator.auxiliary_functions as auxfuncs

In [14]:
reload(tqd)

<module 'Two_qubits_design_PP_qubits' from 'C:\\Python\\Github\\QCreator\\designs\\Two_qubits_design_pp_qubits\\Two_qubits_design_PP_qubits.py'>

In [15]:
tqd.sample.draw_design()
tqd.sample.watch(True)

C:\Python\Github\QCreator\designs\Two_qubits_design_pp_qubits


# Connect resonators into feedline

In [5]:
tqd.sample.connect_cpw(o1=tqd.p1, o2=tqd.p2, port1='port', port2='port', name='feedline', 
                       points=[(2200,3000),(2200,3300),(6500,3300),(6500,4000),(5000,4000)])

[CPW "feedline", l=9125.310]

In [6]:
tqd.sample.draw_design()
tqd.sample.watch(True)

# Some calculations

In [ ]:
from QCreator.elements import twoqtc_sample

In [ ]:
import numpy as np
from scipy.constants import hbar,h,e,c
from scipy.constants import elementary_charge as elc
F0 = 2.0678e-15

In [ ]:
#to suppress printing
@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [ ]:
e_eff=(11.45+1)/2## for all calculations
Mhz=(2*np.pi*1e6)## divide on it to obtain value in mhz
j=0.5e-3      # critical current density nA/nm^2

In [ ]:
def I(f,Ec):
    Ej=Ejf(f,Ec)
    I=Ej*2*np.pi/F0*h*1e18
    return I
def Ejf(f,Ec):
    Ej=(f+Ec)**2/(8*Ec)
    return Ej
def Eji(I):#nA
    return I*F0/(2*np.pi)/1e9/h/1e9
def IEj(Ej):
    return Ej/(F0/(2*np.pi)/1e9/h/1e9)
def f(Ej,Ec):
    return np.sqrt(8*Ej*Ec)-Ec

def Ec(C):#fF
    return (elc)**2/(2*C)/h/1e9*1e15

def cE(Ec):
    return elc**2/(2*Ec)/h/1e9*1e15

#C_sum from Capacitance matrix (0 is ground )
def C_sum(C,i):
    C_SUM =  -C[2*i+1][2*i+2]
    for j in [0,1,2,3,4,5,6]:
        if j == 2*i+1 or j == 2*i+2:
            continue
        C_SUM += -(C[2*i+1][j]*C[2*i+2][j])/(C[2*i+1][j]+C[2*i+2][j])
    return C_SUM

#C_c from Capacitance matrix (0 is ground)
def C_c(C,i,j):
    #print([2*i+2],[2*j+1])
    C_C =  -C[2*i+1][2*j+1]
    C_C += C[2*i+2][2*j+1]
    C_C += -C[2*i+2][2*j+2]
    C_C += C[2*i+1][2*j+2] 
    for k in (2*i+1,2*i+2):
        for l in (2*j+1,2*j+2):
            if l == 2*j+2:
                continue
            C_C += -(C[0][k]*C[0][l])/(C[0][k]+C[0][l])*(-1)**(l+k) #checking the sign
    #print('C_C',C_C)
    return -C_C


def C_c_Stefan(C,i,j):
    Cisum = C_sum(C,i)
    Cjsum = C_sum(C,j)
    i1,i2 = (2*i+1),(2*i+2)
    j1,j2 = (2*j+1),(2*j+2)
    
    denominator = ((C[i1][j1]+C[i1][j2]+C[i2][j1]+C[i2][j2])**2+2*(C[j1][j2]-Cjsum)*(-2*C[i1][i2]+Cisum+Cjsum))
    
    nominator = -(C[i1][j2])**2*C[i2][j1]+(C[i1][j1])**2*C[i2][j2]+(C[i2][j1]-C[i2][j2])*(C[i1][i2]-Cisum)*(C[j1][j2]-Cjsum)-C[i1][j2]*(C[i2][j1]*(C[i2][j1]+C[i2][j2])+(C[i1][i2]-Cisum)*(-C[j1][j2]+Cjsum))
    
    nominator+= C[i1][j1]*(C[i1][j2]*(-C[i2][j1]+C[i2][j2])+C[i2][j2]*(C[i2][j1]+C[i2][j2])+(C[i1][i2]-Cisum)*(-C[j1][j2]+Cjsum))
          
    C_c = nominator/denominator
    
    return C_c

"""

    (−〖C1122〗^2⁢C1221+〖C1121〗^2⁢C1222+(C1221−C1222)⁢(C1112−Cq1Σ)⁢(C2122−Cq2Σ)
−C1122⁢(C1221⁢(C1221+C1222)+(C1112−Cq2Σ)⁢(−C2122+Cq2Σ))
+C1121⁢(C1122⁢(−C1221+C1222)+C1222⁢(C1221+C1222)+(C1112−Cq2Σ)⁢(−C2122+Cq2Σ)))  
/ (C1121+C1122+C1221+C1222)^2+2⁢(C2122−Cq2Σ)⁢(−2⁢C1112+Cq1Σ+Cq2Σ)

"""
    
    
    

def g(Ej1,Ej2,Ec1,Ec2,Ecc):
    return pow(Ej1*Ej2/(4*Ec1*Ec2),0.25)*2*Ec1*Ec2/Ecc


def g_2(w1,w2,C1,C2,Cc):
    return 0.5*np.sqrt(w1*w2)*Cc/np.sqrt(C1*C2)



In [ ]:
###############################################
#Set desired values here in GHz/2pi
w1 = 6.2
U1 = -0.295

w2 = 5.9
U2 = -0.2

wtc= 8.1
Utc= -0.1758
d  = None #defined as (y-1)/(y+1)
y  = 17/8

g1 = 0.142
g2 = 0.116

g12= 0.00905
###############################################

In [ ]:
Ec1 = -U1
Ej1 = Ejf(w1,Ec1)

Ec2 = -U2
Ej2 = Ejf(w2,Ec2)

#TC

Ectc = -Utc
Ejtc = Ejf(wtc,Ectc)


Ic1   = IEj(Ej1)
jj_a1 = np.sqrt(Ic1/j)*1e-3

Ic2   = IEj(Ej2)
jj_a2 = np.sqrt(Ic2/j)*1e-3

print(jj_a2)

In [ ]:
#general settings
center=(1750,1750)
name = 'Martinis_Design'
layer_configuration={
    'total':0,
    'restricted area':10,
    'for removing':100,
    'JJs':1,
    'flux lines':6,
    'air bridges':2,
    'air bridge pads':3,
    'vertical gridlines':15,
    'horizontal gridlines':16,
    'inverted':999
}

l, t_m, t_r, gap, l_arm, h_arm, s_gap = 100, 12, 6, 5, 20, 50, 5
fluxline = {'l':l,'t_m':t_m,'t_r':t_r,'gap':gap,'l_arm':l_arm,'h_arm':h_arm,'s_gap':s_gap}

Couplers = []


# initial settings
g_w   = 310
g_h   = 650
g_t   = 10
b_g   = 10 # for JJ_manhatten
b_w   = 16 # JJ design



width = [75,75,75]
height= [550,550,300]
gap   = [70,70,70]


a1    = jj_a1 #Junction height in um
a2    = a1 # Junction width in um

#jj_pp = { 'a1':a1,"a2":a2,'angle_JJ':np.pi/2}
jj_pp = { 'a1':a1,"a2":a2,'angle_JJ':0,'manhatten':True,'h_w':5 ,'h_d':8 }



#,'bridge_gap':bridge_gap
Q1 = {'name':'Q1','width' : width[0],'height':height[0],'b_g':b_g,'b_w':b_w,'gap':gap[0],'g_w':g_w,'g_h':g_h,'g_t':g_t,
     'jj_pp' : jj_pp, 'Couplers': Couplers, 'transformations':{},'empty_ground':0.66
     }


Q2 = Q1.copy()
Q2['name'] = 'Q2'



TC = Q1.copy()
TC['fluxline'] = fluxline
TC['name'] = 'TC'
TC['g_w'] = 750
TC['g_h'] = 15/7*200
TC['gap'] = 1.9/7*200
TC['width'] = 2.1/7*200
TC['height'] = 11/7*200

lph,lg,lpw,lw = 85,0,55,0.5*200/7
rph,rg,rpw,rw = lph,lg,lpw,lw

TC['arms'] = {'l.ph':lph,'l.g':lg,'l.pw':lpw,'l.w':lw,'r.ph':rph,'r.g':rg,'r.pw':rpw,'r.w':rw}

#Q1['width'],Q1['height'],Q1['gap'] = 308,670,275

d1,d2,mesh_volume = 0,0,50

print(Q1)

In [ ]:
d1,d2,mesh_volume = 0,0,50
C_Matrix = twoqtc_sample.twoqtc_Caps(name,layer_configuration,center,Q1,Q2,TC,d1,d2,mesh_volume,plot = True)

In [ ]:
print(C_Matrix/6.225)

In [ ]:
print(C_Matrix)

In [ ]:
C_Matrix2 = np.array([[0.043071,-0.0089447,-0.0068652,-0.0068713,-0.0089472,-0.0056849,-0.0057579],[-0.0089447,0.015124,-0.0047549,-0.00023452,-0.00023108,-0.00067197,-0.00028649],[-0.0068652,-0.0047549,0.014569,-0.00024721,-0.00023463,-0.0020928,-0.0003746],[-0.0068713,-0.00023452,-0.00024721,0.01457,-0.0047545,-0.0003784,-0.0020844],[-0.0089472,-0.00023108,-0.00023463,-0.0047545,0.015124,-0.00028975,-0.00066677],[-0.0056849,-0.00067197,-0.0020928,-0.0003784,-0.00028975,0.012224,-0.0031063],[-0.0057579,-0.00028649,-0.0003746,-0.0020844,-0.00066677,-0.0031063,0.012276]])
C_Matrix2 = 6225*C_Matrix2
print(C_Matrix2)

## Optimization 1 for Qubit Parameters

In [ ]:
#Calculate values for w from here
Ec1_sim = Ec(C_sum(C_Matrix,0))
Ec2_sim = Ec(C_sum(C_Matrix,1))
Ectc_sim = Ec(C_sum(C_Matrix,2))

w1_sim = f(Ej1,Ec1_sim)
w2_sim = f(Ej2,Ec2_sim)
wtc_sim = f(Ejtc,Ectc_sim)


print(Ec1_sim,Ec2_sim,Ectc_sim)

print(w1_sim,w2_sim,wtc_sim)

In [ ]:
#optimization parameters
tol   = 0.001 # GHz
mesh_v = 5

width_steps  = [-3,0,2,3,4,5,6,7,8,9,10,12,15]
height_steps = [-3,0,2,2.5,2.9,3,3.2,3.3,4,5,6,7,8]
gap_steps    = [-3,0,2,3,4,5,6,7,8,9,10,12,15]

#from here change values to get correct qubit values in a loop 
d_Ec = [Ec1 - Ec1_sim, Ec2 - Ec2_sim,Ectc - Ectc_sim]
d_w = [w1 - w1_sim, w2 - w2_sim,wtc - wtc_sim]
max_iter = 20
k = 1
while np.sum(np.abs(d_w)) > 3*tol and k < max_iter:
    Ec1_sim = Ec(C_sum(C_Matrix,0))
    Ec2_sim = Ec(C_sum(C_Matrix,1))
    Ectc_sim = Ec(C_sum(C_Matrix,2))

    w1_sim = f(Ej1,Ec1_sim)
    w2_sim = f(Ej2,Ec2_sim)
    wtc_sim = f(Ejtc,Ectc_sim)
    d_w = [w1 - w1_sim, w2 - w2_sim,wtc - wtc_sim]
    d_Ec = [Ec1 - Ec1_sim, Ec2 - Ec2_sim,Ectc - Ectc_sim]
    print('__________________________________________')
    print('Pass '+str(k)+' Adjusting Qubit parameters')
    print('Current Error: \033[93m' + str(np.sum(np.abs(d_w))) + ' GHz\033[0m')
    mesh_volume = mesh_v
    if np.sum(np.abs(d_w)) > 0.1:
        print('Using coarse mesh')
        mesh_volume = 40
    else:
        print('Refining mesh')
    for i in [0,1,2]:
        d = d_Ec[i]
        if np.sum(np.abs(d_w[i])) > 0.5:
            width_step,height_step,gap_step = np.random.choice(width_steps[-5:]),np.random.choice(height_steps[-5:]),np.random.choice(gap_steps[-5:])
        else:
            width_step,height_step,gap_step = np.random.choice(width_steps[:10]),np.random.choice(height_steps[:10]),np.random.choice(gap_steps[:10])

        if np.abs(d) < tol*0.01:
            continue
        if d > 0 :
            width[i]  += -width_step
            height[i] += -height_step
            gap[i]    += gap_step
        if d < 0 :
            width[i]  += width_step
            height[i] += height_step
            gap[i]    += -gap_step
    print('Running Fastcap simulation')
    Q1['width'] = width[0]
    Q1['height']= height[0]
    Q1['gap']= gap[0]
 
    Q2['width'] = width[1]
    Q2['height']= height[1]
    Q2['gap']= gap[1]

    TC['width'] = width[2]
    TC['height']= height[2]
    TC['gap']= gap[2]
    with suppress_stdout():
        C_Matrix = twoqtc_sample.twoqtc_Caps(name,layer_configuration,center,Q1,Q2,TC,d1,d2,mesh_volume)
    k += 1
print(Q1)
print(Q2)
print(TC)

In [ ]:
#Calculate values for w from here
Ec1_sim = Ec(C_sum(C_Matrix,0))
Ec2_sim = Ec(C_sum(C_Matrix,1))
Ectc_sim = Ec(C_sum(C_Matrix,2))

w1_sim = f(Ej1,Ec1_sim)
w2_sim = f(Ej2,Ec2_sim)
wtc_sim = f(Ejtc,Ectc_sim)

print(w1_sim,w2_sim,wtc_sim)
print(w1,w2,wtc)

In [ ]:
print(Q1)
twoqtc_sample.twoqtc_Caps(name,layer_configuration,center,Q1,Q2,TC,d1,d2,mesh_volume,save = True)

## Optimization 2 for Coupling Parameters

In [ ]:
Ecc12_sim = Ec(C_c(C_Matrix,0,1))
Ecc1tc_sim = Ec(C_c(C_Matrix,0,2))

print(Ecc12_sim,Ecc1tc_sim)

In [ ]:
C_c(C_Matrix,0,2)

In [ ]:
#fastcap matrix
w1_sim = 4.973 
w2_sim = 5.163
wtc_sim= 7.18

Ec1_sim = Ec(C_sum(C_Matrix,0))
Ec2_sim = Ec(C_sum(C_Matrix,1))
Ectc_sim=Ec(C_sum(C_Matrix,2))


Ej1 = Ejf(w1_sim,Ec1_sim) 
Ej2 = Ejf(w2_sim,Ec2_sim)
Ejtc = Ejf(wtc_sim,Ectc_sim)



Cc = C_c(C_Matrix,0,2)
Cc2= C_c_Stefan(C_Matrix,0,2)

print('C_c from equ1 : ',C_c(C_Matrix,0,2),'fF')
print('C_c from equ2 : ',C_c_Stefan(C_Matrix,0,2),'fF')

print('C_sum1             C_sum2            C_sumtc')
print(C_sum(C_Matrix,0),C_sum(C_Matrix,1),C_sum(C_Matrix,2))
print('Ec1                Ec2               Ectc')
print(Ec1_sim,Ec2_sim,Ectc_sim)

print('------')
#(Ej1,Ej2,Ec1,Ec2,Ecc)
print(g(Ej1,Ejtc,Ec1_sim,Ectc_sim,Ec(Cc2)))
print(g_2(w1_sim,wtc_sim,C_sum(C_Matrix,0),C_sum(C_Matrix,2),Cc2))

In [ ]:
#hfss Matrix
w1_sim = 4.973 
w2_sim = 5.163
wtc_sim= 7.18

Ec1_sim = Ec(C_sum(C_Matrix2,0))
Ec2_sim = Ec(C_sum(C_Matrix2,1))
Ectc_sim=Ec(C_sum(C_Matrix2,2))


Ej1 = Ejf(w1_sim,Ec1_sim) 
Ej2 = Ejf(w2_sim,Ec2_sim)
Ejtc = Ejf(wtc_sim,Ectc_sim)


Cc = C_c(C_Matrix2,0,2)
Cc2= C_c_Stefan(C_Matrix2,0,2)

print('C_c from equ1 : ',C_c(C_Matrix2,0,2),'fF')
print('C_c from equ2 : ',C_c_Stefan(C_Matrix2,0,2),'fF')

print('C_sum1             C_sum2            C_sumtc')
print(C_sum(C_Matrix2,0),C_sum(C_Matrix2,1),C_sum(C_Matrix2,2))
print('Ec1                Ec2               Ectc')
print(Ec1_sim,Ec2_sim,Ectc_sim)

print('------')
#(Ej1,Ej2,Ec1,Ec2,Ecc)
print(g(Ej1,Ejtc,Ec1_sim,Ectc_sim,Ec(Cc2)))
print(g_2(w1_sim,wtc_sim,C_sum(C_Matrix2,0),C_sum(C_Matrix2,2),Cc2))

In [ ]:
g12_sim = g(Ej1,Ej2,Ec1_sim,Ec2_sim,Ec(C_c(C_Matrix,0,1)))
g1_sim = g(Ej1,Ejtc,Ec1_sim,Ectc_sim,Ec(C_c(C_Matrix,0,2)))
g2_sim = g(Ej2,Ejtc,Ec2_sim,Ectc_sim,Ec(C_c(C_Matrix,1,2)))
print(g12_sim,g1_sim,g2_sim)

In [ ]:
pad = [50,50]
g_w = [1000,1000,1500]
#optimization parameters
tol   = 0.01 # GHz
mesh_v = 10

distance_steps  = [-5,0,5,10,15,20,25,30,40,50]


#from here change values to get correct qubit values in a loop 
d_g = [g1 - g1_sim, g2 - g2_sim,g12 - g12_sim]

max_iter = 15
k = 1
while np.sum(np.abs(d_g)) > tol*3 and k < max_iter:
    Ec1_sim = Ec(C_sum(C_Matrix,0))
    Ec2_sim = Ec(C_sum(C_Matrix,1))
    Ectc_sim = Ec(C_sum(C_Matrix,2))
    
    g12_sim = g(Ej1,Ej2,Ec1_sim,Ec2_sim,Ec(C_c(C_Matrix,0,1)))
    g1_sim = g(Ej1,Ejtc,Ec1_sim,Ectc_sim,Ec(C_c(C_Matrix,0,2)))
    g2_sim = g(Ej2,Ejtc,Ec2_sim,Ectc_sim,Ec(C_c(C_Matrix,1,2)))
    d_g = [g1 - g1_sim, g2 - g2_sim,g12 - g12_sim]
    print(d_g)
    
    print('__________________________________________')
    print('Pass '+str(k)+' Adjusting Qubit coupling parameters')
    print('Current Error: \033[93m' + str(1000*np.sum(np.abs(d_g))) + ' MHz\033[0m')
    mesh_volume = mesh_v
    if np.sum(np.abs(d_g)) > 0.1:
        print('Using coarse mesh')
        mesh_volume = 60
    else:
        print('Refining mesh')
    
    for i in [0,1,2]:
        d = d_g[i]
        if np.sum(np.abs(d_g[i])) > 0.5:
            d_step = np.random.choice(distance_steps[-5:])
            height_step,gap_step = np.random.choice(height_steps[-5:]),np.random.choice(gap_steps[-5:])
        else:
            d_step = np.random.choice(distance_steps[:10])
            height_step,gap_step = np.random.choice(height_steps[:10]),np.random.choice(gap_steps[:10])
        if np.abs(d) < tol*0.01:
            continue
        if i == 0:
            if d < 0 :
                pad[0]  -= height_step
            if d > 0 :
                pad[0]  += height_step            
        if i == 1:
            if d < 0 :
                pad[1]  -= height_step
            if d > 0 :
                pad[1]  += height_step             
        if i ==2:
            if d < 0 :
                g_w[2]  += d_step
            if d > 0 :
                g_w[2]  -= d_step
            
    print('Running Fastcap simulation')
    Q1['g_w'] = g_w[0]
    Q2['g_w'] = g_w[1]
    TC['g_w'] = g_w[2]

    
    Q1['height']= height[0]
    Q1['gap']= gap[0]
 
    Q2['width'] = width[1]
    Q2['gap']= gap[1]

    TC['width'] = width[2]
    TC['gap']= gap[2]
    TC['arms'] = {'l.ph':pad[0],'l.g':lg,'l.pw':lpw,'l.w':lw,'r.ph':pad[1],'r.g':rg,'r.pw':rpw,'r.w':rw}
    
    with suppress_stdout():
        C_Matrix = twoqtc_sample.twoqtc_Caps(name,layer_configuration,center,Q1,Q2,TC,d1,d2,mesh_volume)
    k += 1
    
    
print('Simulation finished')

In [ ]:
g12_sim = g(Ej1,Ej2,Ec1_sim,Ec2_sim,Ec(C_c(C_Matrix,0,1)))
g1_sim = g(Ej1,Ejtc,Ec1_sim,Ectc_sim,Ec(C_c(C_Matrix,0,2)))
g2_sim = g(Ej2,Ejtc,Ec2_sim,Ectc_sim,Ec(C_c(C_Matrix,1,2)))


g1 - g1_sim, g2 - g2_sim,g12 - g12_sim
print(g1,g2,g12)
print(g1_sim,g2_sim,g12_sim)


#Calculate values for w from here
Ec1_sim = Ec(C_sum(C_Matrix,0))
Ec2_sim = Ec(C_sum(C_Matrix,1))
Ectc_sim = Ec(C_sum(C_Matrix,2))

w1_sim = f(Ej1,Ec1_sim)
w2_sim = f(Ej2,Ec2_sim)
wtc_sim = f(Ejtc,Ectc_sim)

print(w1_sim,w2_sim,wtc_sim)
print(w1,w2,wtc)

In [ ]:
twoqtc_sample.twoqtc_Caps(name,layer_configuration,center,Q1,Q2,TC,d1,d2,mesh_volume,plot = True)

In [ ]:
print(((0,None),)*11)